<a href="https://colab.research.google.com/github/Forbusinessuseyukikoishiguro/---PDF-20250403/blob/main/_%E3%80%90note%E5%85%AC%E9%96%8B%E7%94%A8%E3%80%91Google%E3%82%B3%E3%83%A9%E3%83%9C%E7%89%88%E3%81%AB%E3%82%83%E3%82%93%E3%81%93%E3%83%81%E3%83%A3%E3%83%83%E3%83%88%E3%83%9C%E3%83%83%E3%83%882025_04_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##【note公開用】Googleコラボ版にゃんこチャットボット

ーーーーーーーーーーーーーーーーー
##基本の操作方法
##①実行：▶をクリックしてください。
##②APIキーを四角い枠の中に入力してください。
##③PDFをインポートしてください。
ーーーーーーーーーーーーーーーーー
##その他
####ご利用にはOPENAI_API_KEYの設定が必要です。APIキーのご入手をお願い致します。
####OpenAIのAPIの仕様が大幅にアップデートされた場合、動かなくなることがございます。
####動作不良の場合はご連絡いただけますと幸いです。
####2025/04/05時点動作確認ずみ



In [ ]:

# にゃんこ先生のPDF Q&Aシステム（Google Colab版）
# このノートブックはPDFファイルに対して質問をするにゃん♪
# 必要なライブラリのインストールからPDF分析までを自動化してるにゃ～


# @title コード
!pip install -q llama-index openai pypdf langchain


# @title APIキー設定とライブラリのインポートにゃ～
import os
import sys
from pathlib import Path
import traceback
from google.colab import files
import pandas as pd
from IPython.display import display, HTML, Markdown


# APIキーを設定するセルにゃん
from getpass import getpass
import os


# @markdown APIキーを入力するにゃ～（入力は表示されないにゃ）
api_key = getpass('OpenAI API Key にゃん: ')
os.environ["OPENAI_API_KEY"] = api_key


# APIキーの最初の数文字を表示して確認するにゃ
if api_key and api_key.startswith('sk-'):
    print(f"APIキー確認したにゃん！ちゃんと設定されてるにゃ (先頭: {api_key[:5]}...)")
else:
    print("にゃにゃ？APIキーの形式が正しくない可能性があるにゃ...")

# @title PDFファイルのアップロードにゃん
# PDFファイルをアップロードするセルにゃ
print("PDFファイルをアップロードするにゃん...")
uploaded = files.upload()


# アップロードされたファイル名を取得するにゃ
pdf_filenames = list(uploaded.keys())


if not pdf_filenames:
    print("にゃぁ～ファイルがアップロードされてないにゃ...")
else:
    print(f"アップロードされたファイルを見つけたにゃん: {pdf_filenames}")


# @title PDF分析の準備をするにゃん
# @markdown PDFファイルを選択し、最初の質問を入力するにゃ！


# @markdown ### ファイルと設定にゃん
# @markdown PDFファイルを選択するにゃ (アップロード済みのファイルから)
pdf_file = pdf_filenames[0] if pdf_filenames else "" # @param {type:"string"}
# @markdown 使用するモデルにゃ～
model = "gpt-4" # @param ["gpt-3.5-turbo", "gpt-4", "gpt-4-turbo"]
# @markdown 温度パラメータ (0.0～1.0, 低いほど一貫性が高いにゃ)
temperature = 0.1 # @param {type:"slider", min:0, max:1, step:0.1}


# @markdown ### 質問内容にゃん
# @markdown 最初の質問を入力するにゃん
query = "このドキュメントの主要ポイントを教えて" # @param {type:"string"}


# PDFファイルのパスを設定するにゃん
pdf_path = Path(pdf_file) if pdf_file else None


def analyze_pdf(pdf_path, model, temperature, query):
    """PDFを分析して質問に回答する関数にゃん"""

    if not pdf_path or not pdf_path.exists():
        print(f"にゃぁ～！PDFファイル '{pdf_path}' が見つからないにゃ...")
        return

    try:
        # 最新バージョンのllama-indexを使用するにゃん
        print("llama-indexを初期化中...にゃん")

        # 必要なライブラリのインポートにゃ
        from llama_index.core import VectorStoreIndex, Settings
        from llama_index.core.readers import SimpleDirectoryReader
        from llama_index.llms.openai import OpenAI
        from llama_index.core import StorageContext

        # PDFリーダーの設定にゃん
        reader = SimpleDirectoryReader(input_files=[str(pdf_path)])

        # 指定されたモデルを使用するにゃ～
        llm = OpenAI(model=model, temperature=temperature)

        # 最新の設定インターフェースを使用にゃん
        Settings.llm = llm
        Settings.chunk_size = 1024

        print(f"使用モデル: {model}, 温度: {temperature}にゃん")

        # PDFの読み込みにゃん
        print(f"PDFファイル '{pdf_path}' を読み込み中...にゃん")
        documents = reader.load_data()
        print(f"PDFの読み込み完了したにゃ！ {len(documents)} セクションに分割したにゃ")

        # インデックスの作成とキャッシュディレクトリにゃん
        storage_dir = f"./storage_{pdf_path.stem}_{model.replace('-', '_')}/"

        # キャッシュが存在するか確認するにゃん
        has_cache = Path(storage_dir).exists()

        if has_cache:
            print(f"既存のインデックスを見つけたにゃん: {storage_dir}")
            print("保存済みインデックスを読み込むにゃ...")

            # インデックスの読み込みにゃん
            storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
            index = VectorStoreIndex.from_storage(storage_context)
        else:
            print("インデックスが見つからないから新しく作るにゃん...")
            print("テキストをベクトル化中...にゃん")
            index = VectorStoreIndex.from_documents(documents)

            # インデックスの保存にゃん
            os.makedirs(storage_dir, exist_ok=True)
            index.storage_context.persist(persist_dir=storage_dir)
            print(f"インデックスを '{storage_dir}' に保存したにゃん！")

        # クエリエンジンの作成にゃん
        print("クエリエンジンを初期化中...にゃん")
        query_engine = index.as_query_engine()

        # 質問の実行にゃん
        print(f"\n質問: {query}にゃん")
        print("\n回答を生成中...にゃん")
        response = query_engine.query(query)

        # 結果の表示にゃん
        display(Markdown(f"## にゃんこ先生の回答だにゃん！\n{response}"))

        # ソース表示にゃん
        print("\n参照元情報にゃん:")
        if hasattr(response, "source_nodes") and response.source_nodes:
            source_df = pd.DataFrame([
                {
                    "参照元": f"参照元 {i+1}にゃ",
                    "テキスト": (source.node.get_content()[:300] + "...") if hasattr(source.node, "get_content") else "内容を表示できないにゃ"
                }
                for i, source in enumerate(response.source_nodes)
            ])
            display(source_df)
        else:
            print("参照元情報が見つからないにゃ...")

        print("\n処理が完了したにゃん！")
        return response, query_engine

    except Exception as e:
        print(f"にゃにゃにゃ！エラーが発生したにゃん: {e}")
        traceback.print_exc()

        print("\n問題解決のヒントにゃん:")
        print("1. APIキーが正しく設定されているか確認するにゃ")
        print("2. PDFファイルが正しく配置され、アクセスできるか確認するにゃん")
        print("3. モデル名が正しいか確認するにゃ")
        return None, None


# PDFファイルがアップロードされている場合のみ実行するにゃん
if pdf_path and pdf_path.exists():
    result, engine = analyze_pdf(pdf_path, model, temperature, query)
else:
    print("PDFファイルをアップロードして、ファイル名を選択するにゃ")
    result, engine = None, None


# @title にゃんこ先生に追加質問をするにゃん！
# @markdown 質問を入力すると、にゃんこ先生がPDFを参照して回答するにゃ～


# クエリエンジンがある場合のみ表示するにゃん
if 'engine' in locals() and engine is not None:
    # @markdown 質問を入力するにゃん
    follow_up_query = "" # @param {type:"string"}

    if follow_up_query:
        print(f"\n追加質問: {follow_up_query}にゃん")
        print("考え中...にゃん...")
        response = engine.query(follow_up_query)

        # 結果の表示にゃん
        display(Markdown(f"## にゃんこ先生の回答だにゃん！\n{response}"))

        # ソース表示にゃん
        print("\n参照元情報にゃん:")
        if hasattr(response, "source_nodes") and response.source_nodes:
            source_df = pd.DataFrame([
                {
                    "参照元": f"参照元 {i+1}にゃ",
                    "テキスト": (source.node.get_content()[:300] + "...") if hasattr(source.node, "get_content") else "内容を表示できないにゃ"
                }
                for i, source in enumerate(response.source_nodes)
            ])
            display(source_df)
        else:
            print("参照元情報が見つからないにゃ...")
else:
    print("最初にPDFの分析が完了してからにゃ～")


# @title にゃんこ先生との対話モードにゃん！
# @markdown 複数の質問をまとめて入力できるにゃ～（一行ずつ書くにゃ）
# クエリエンジンがある場合のみ表示するにゃん
if 'engine' in locals() and engine is not None:
    # @markdown 質問リスト（一行に一つの質問を書くにゃ）
    multiple_queries = "" # @param {type:"string"}

    if multiple_queries:
        # 質問を行ごとに分割するにゃん
        query_list = [q.strip() for q in multiple_queries.split('\n') if q.strip()]

        for i, q in enumerate(query_list):
            print(f"\n===== 質問 {i+1}: {q} =====")
            print("考え中...にゃん...")
            response = engine.query(q)

            # 結果の表示にゃん
            display(Markdown(f"## にゃんこ先生の回答 {i+1} だにゃん！\n{response}"))

            # ソース表示にゃん（簡易版）
            if hasattr(response, "source_nodes") and response.source_nodes:
                print(f"参照元: {len(response.source_nodes)}箇所見つけたにゃん！")
            else:
                print("参照元情報が見つからないにゃ...")
else:
    print("最初にPDFの分析が完了してからにゃ～")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 10.6 MB/s eta 0:00:00
OpenAI API Key にゃん: ··········
APIキー確認したにゃん！ちゃんと設定されてるにゃ (先頭: sk-pr...)
PDFファイルをアップロードするにゃん...


Saving 健康のすべて.pdf to 健康のすべて.pdf
アップロードされたファイルを見つけたにゃん: ['健康のすべて.pdf']
llama-indexを初期化中...にゃん
使用モデル: gpt-4, 温度: 0.1にゃん
PDFファイル '健康のすべて.pdf' を読み込み中...にゃん
PDFの読み込み完了したにゃ！ 13 セクションに分割したにゃ
インデックスが見つからないから新しく作るにゃん...
テキストをベクトル化中...にゃん
インデックスを './storage_健康のすべて_gpt_4/' に保存したにゃん！
クエリエンジンを初期化中...にゃん

質問: このドキュメントの主要ポイントを教えてにゃん

回答を生成中...にゃん


## にゃんこ先生の回答だにゃん！
このドキュメントは、健康とその維持についての様々な側面を探求しています。まず、健康は私たちが生活する環境に大きく影響を受け、環境保護や持続可能な開発が重要であることを強調しています。次に、積極的な健康への取り組み、定期的な健康チェックやスクリーニング、バランスの取れた食事や適度な運動、ストレスの管理、適切な睡眠が健康維持に不可欠であることを説明しています。さらに、健康への意識向上と啓発活動が社会全体の健康づくりに重要な役割を果たすこと、そして健康に関する資源と情報の活用が重要であることを述べています。

また、疾病の予防が健康維持のために重要であり、予防医学の手法として予防接種やスクリーニングテスト、健康診断の受診などがあることを指摘しています。環境への配慮が健康への取り組みにおいて重要であること、科学技術の進歩が健康分野においても革新的な進展をもたらしていること、そしてアートやクリエイティビティが健康に対してポジティブな影響を与えることも強調しています。


参照元情報にゃん:


,参照元,テキスト
0,参照元 1にゃ,５．健康と環境 \n \n私たちの健康は、 私たちが生活する環境にも大きく影響を受けます。 ...
1,参照元 2にゃ,健康を維持するためには、疾病の予防が重要です。予防医学の手法としては、予防接種やス\nクリー...



処理が完了したにゃん！
